In [42]:
from collections import deque
import heapq

In [2]:
g = "123804765"
s1 = "134862705"
s2 = "281043765"
s3 = "567408321"

In [3]:
previous_states = set()

In [4]:
def print_state(state:str):
    arr = [state[a:a+3] for a in (0,3,6)]
    for a in arr:
        print(" ".join(a))

In [5]:
def successor(state:str):
    index = state.index("0")
    i = index//3
    j = index%3

    next_moves = []
    #move up
    if i > 0:
        #convert the string to a list to make it mutable
        temp_state = list(state)
        
        #switch the values in the list
        temp_state[index], temp_state[index-3] = temp_state[index-3], temp_state[index]
        
        #the cost of the move will be the value of the new item at index
        cost = int(temp_state[index])
        
        #join the new state to get the string representation back
        new_state = "".join(temp_state)
        
        #check if the state was previously visited
        if new_state not in previous_states:
            #keep the cost the first item in the tuple to be used in a heapq object
            next_moves.append((cost,new_state,"UP"))
    #move down
    if i < 2:
        temp_state = list(state)
        temp_state[index], temp_state[index+3] = temp_state[index+3], temp_state[index]
        cost = int(temp_state[index])
        new_state = "".join(temp_state)
        if new_state not in previous_states:
            next_moves.append((cost,new_state,"DOWN"))
    #move left
    if j > 0:
        temp_state = list(state)
        temp_state[index], temp_state[index-1] = temp_state[index-1], temp_state[index]
        cost = int(temp_state[index])
        new_state = "".join(temp_state)
        if new_state not in previous_states:
            next_moves.append((cost,new_state,"LEFT"))
    #move right
    if j < 2:
        temp_state = list(state)
        temp_state[index], temp_state[index+1] = temp_state[index+1], temp_state[index]
        cost = int(temp_state[index])
        new_state = "".join(temp_state)
        if new_state not in previous_states:
            next_moves.append((cost,new_state,"RIGHT"))
    
    return next_moves

In [20]:
class Node:
    def __init__(self,state:str,total_cost:int,path_length:int,move:str=None,predecessor=None):
        #current state of the node
        self.state = state
        
        #total cost to get to the node
        self.total_cost = total_cost
        
        #depth of the node in the search tree with root as 0
        self.path_length = path_length
        
        #the move taken to reach the node from the previous node
        self.move = move
        
        #the predecessor Node object
        #traverse the predecessor links to get the solution
        self.predecessor = predecessor
        
    #need lt function to allow for heap pop
    def __lt__(self,n):
        return self.total_cost < n.total_cost

In [21]:
def get_solution(n:Node):
    #append all the connected nodes in the path to the list
    #uncomment the last append to include the root
    solution = []
    while n.predecessor:
        solution.append(n)
        n = n.predecessor
    #solution.append(n)
    return solution

In [22]:
def in_path(state:str,n:Node):
    while n.predecessor:
        if state == n.state:
            return True
        n = n.predecessor
    #check the root node, it wont be in the loop due to predecessor being None
    if state == n.state:
        return True
    
    return False

In [23]:
def h_count(state:str,goal:str):
    return sum([a==b for a,b in zip(state,goal)])

In [47]:
def h_dist(state:str,goal:str):
    dist = 0
    for index,char in enumerate(state):
        i1 = index//3
        j1 = index%3
        
        index_g = goal.index(char)
        i2 = index_g//3
        j2 = index_g%3
        
        dist += abs(i1-i2)+abs(j1-j2)
    return dist

In [126]:
def h_dist_cost(state:str,goal:str):
    dist = 0
    for index,char in enumerate(state):
        i1 = index//3
        j1 = index%3
        
        index_g = goal.index(char)
        i2 = index_g//3
        j2 = index_g%3
        
        dist += int(char) * (abs(i1-i2)+abs(j1-j2))
    return dist

In [118]:
def BFS(root:str,goal:str):
    max_q_size = 0
    #testing var for queue size
    q_size = 1
    #testing var for number of nodes popped
    time = 0
    
    #dumb idea using a global set. have to clear each time i run.
    #should I refactor the code to make it a function level set? would probably have to pass it as an arg each time though
    previous_states.clear()
    
    #using a queue to store the ordering
    q = deque()
    
    #add the root to the queue
    q.append(Node(root,0,0))
    
    while q:
        if q_size > max_q_size:
            max_q_size = q_size
        
        #pop the top of the queue to get the current item to work on
        #not pop, use popleft since we are appending right
        current = q.popleft()
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        time += 1
        #check if at the goal
        if current.state == goal:
            return current, max_q_size, time
        else:
            #add the state to the checked states set
            previous_states.add(current.state)
            
            #get the successors
            children = successor(current.state)
            for c in children:
                q.append(Node(c[1], current.total_cost + c[0], current.path_length + 1, c[2], current))
                q_size += 1
            

In [119]:
#TODO: limit the function runtime to n minutes


#getting same results as BFS. why?
#resolved. was appending to wrong side in while loop
def DFS(root:str,goal:str):
    max_q_size = 0
    q_size = 1
    time = 0
    
    previous_states.clear()
    
    q = deque()
    
    #now using stack instead of queue, so append
    q.append(Node(root,0,0))
    
    while(q):
        if q_size > max_q_size:
            max_q_size = q_size
        
        #stack so pop from same side as append
        current = q.pop()
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        time += 1
        
        if current.state == goal:
            return current, max_q_size, time
        else:
            previous_states.add(current.state)
            
            children = successor(current.state)
            for c in children:
                #stupid mistake. was using append left before, not append. Should be fixed now.
                q.append(Node(
                    c[1], current.total_cost + c[0], current.path_length + 1, c[2], current
                ))
                q_size += 1

In [120]:
#cloned from DFS function
def DFS_limited(root:str,goal:str,limit=50):
    max_q_size = 0
    q_size = 1
    time = 0
    
    previous_states.clear()
    
    q = deque()
    
    #now using stack instead of queue, so append
    q.append(Node(root,0,0))
    
    while(q):
        if q_size > max_q_size:
            max_q_size = q_size
        
        #stack so pop from same side as append
        current = q.pop()
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        time += 1
        
        if current.state == goal:
            return current, max_q_size, time
        else:
            previous_states.add(current.state)
            
            if current.path_length > limit:
                continue
            else:
                children = successor(current.state)
                for c in children:
                    q.append(Node(
                        c[1], current.total_cost + c[0], current.path_length + 1, c[2], current
                    ))
                    q_size += 1

In [121]:

def uniform_cost(root:str,goal:str):
    #have to use 2nd q size var
    #q size keeps track of current queue size
    #do max size check at each loop iteration
    max_q_size = 0
    q_size = 1
    time = 0
    
    previous_states.clear()
    
    #use heap for queue
    #heap tracks the total cost of the node traversal
    heap = []
    heapq.heappush(heap,(0,Node(root,0,0)))
    
    while(heap):
        if q_size > max_q_size:
            max_q_size = q_size
            
        cost, current = heapq.heappop(heap)
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        #add time after repeat check since we are not expanding that node
        time += 1
        
        if current.state == goal:
            return current, max_q_size, time
        else:
            #in this case, the expanded nodes for a state is the closest to the root
            #the node will either:
            #1 - expanded early but not part of the solution path. In which case this will prevent further search depth
            #2 - part of the solution path
            #3 - not part of the solution path and will never be expanded
            #allows us to keep the set if we add the expanded nodes to it
            previous_states.add(current.state)
            
            children = successor(current.state)

            for c in children:
                #bit of a change here with the heap. Have to add tuple of (total cost,node)
                heapq.heappush(heap,(current.total_cost + c[0], Node(
                    c[1], current.total_cost + c[0], current.path_length + 1, c[2], current
                )))
                #queue size here is basically all the items in the heap.
                #some states will have repetition, but repeat checking would have linear runtime so not worth doing.
                q_size += 1

In [122]:
#TODO: the solution seems fine, but the space and time are too much
#I think the heuristic function produces too many of the same results (not enough range)

#clone from uniform function. change the value added to the heap tuple
#use a function as an arg to avoid adding the logic in the search function
def best_first(root:str,goal:str,h_func):
    max_q_size = 0
    q_size = 1
    time = 0
    
    previous_states.clear()
    
    heap = []
    heapq.heappush(heap,(0,Node(root,0,0)))
    
    while(heap):
        if q_size > max_q_size:
            max_q_size = q_size
            
        cost, current = heapq.heappop(heap)
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        time += 1
        
        if current.state == goal:
            return current, max_q_size, time
        else:
            #can still use the hash set as the state has a set 
            previous_states.add(current.state)
            
            children = successor(current.state)
            
            for c in children:
                #the heap priority is changed from total cost to the h_func given as an arg
                #h_func gives the heuristic from the child state to the goal
                heapq.heappush(heap,(h_func(c[1],goal), Node(
                    c[1], current.total_cost + c[0], current.path_length + 1, c[2], current
                )))
                q_size += 1

In [123]:
def a_star(root:str,goal:str,h_func):
    max_q_size = 0
    q_size = 1
    time = 0
     
    previous_states.clear()
    
    heap = []
    heapq.heappush(heap,(0,Node(root,0,0)))
    
    while(heap):
        if q_size > max_q_size:
            max_q_size = q_size
        
        cost, current = heapq.heappop(heap)
        q_size -= 1
        
        if current.state in previous_states:
            continue
        
        time += 1
        
        if current.state == goal:
            return current, max_q_size, time
        else:
            previous_states.add(current.state)
            
            children = successor(current.state)
            
            for c in children:
                heapq.heappush(heap,(
                    current.total_cost + c[0] + h_func(c[1],goal), 
                    Node(
                        c[1], current.total_cost + c[0], current.path_length + 1, c[2], current
                    )
                ))
                q_size += 1

In [124]:
def show_results(start:str,goal:str,search,show=True,**kwargs):
    found, q_size, time = search(start,g,**kwargs)
    
    print(f"Time: {time}")
    print(f"Space: {q_size}")
    print(f"Cost: {found.total_cost}")
    print(f"Path length: {found.path_length}")
    print()

    if show:
        solution = get_solution(found)

        print("starting state")
        print_state(start)
        for i in reversed(solution):
            print()
            print(f"{i.move}, total_cost: {i.total_cost}")
            print_state(i.state)

In [133]:
root = s3


print("BFS")
show_results(root, g, BFS,show=False)

print("DFS")
show_results(root, g, DFS,show=False)

limit = 100
try:
    print(f"DFS limited to depth {limit}")
    show_results(root, g, DFS_limited,show=False,limit=limit)
except Exception:
    print(f"unable to find solution with depth limit: {limit}")
    print()
    
print("Uniform cost")
show_results(root, g, uniform_cost,show=False)

print("Best first with misplaced count")
show_results(root, g, best_first, h_func=h_count,show=False)

print("Best first with manhattan distance")
show_results(root, g, best_first, h_func=h_dist,show=False)

print("a* with misplaced count")
show_results(root, g, a_star, h_func=h_count,show=False)

print("a* with manhattan distance")
show_results(root, g, a_star, h_func=h_dist,show=False)

print("a* with manhattan distance*cost")
show_results(root, g, a_star, h_func=h_dist_cost,show=False)

BFS
Time: 181364
Space: 32175
Cost: 132
Path length: 30

DFS
Time: 76102
Space: 50466
Cost: 326102
Path length: 72686

DFS limited
Time: 90731
Space: 83
Cost: 374
Path length: 100

Uniform cost
Time: 181075
Space: 29628
Cost: 128
Path length: 30

Best first with misplaced count
Time: 181440
Space: 41148
Cost: 128
Path length: 30

Best first with manhattan distance
Time: 316
Space: 281
Cost: 252
Path length: 54

a* with misplaced count
Time: 181379
Space: 29158
Cost: 128
Path length: 30

a* with manhattan distance
Time: 169210
Space: 33770
Cost: 128
Path length: 30

a* with manhattan distance*cost
Time: 1194
Space: 626
Cost: 128
Path length: 30



TypeError: can only concatenate str (not "int") to str